In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Filtered_S&P_Stock_Data.csv')
df = df.set_index('Date')
df.head()

,Open,High,Low,Close,Adj Close,Volume,Filtered_Open
Date,,,,,,,
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,1455.219971,931800000,1509.891113
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1399.420044,1009000000,1460.423078
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1402.109985,1085500000,1438.658516
2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1403.449951,1092300000,1386.279016
2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1441.469971,1225200000,1399.262688


In [3]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

Using TensorFlow backend.
C:\Users\daani\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\daani\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\daani\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [4]:
df_train = df.set_index(df.index)[:'2019-05-16']
df_test = df.set_index(df.index)['2019-05-15':]

We will have to scale our value, which would require us to start by using zero-mean normalization on the denoised Open data

In [5]:
df_mean = df_train['Filtered_Open'].mean()

In [6]:
df_mean

1528.9837666632293

In [7]:
df_train['ZeroMeanFilteredOpen'] = df_mean
df_train.head()

,Open,High,Low,Close,Adj Close,Volume,Filtered_Open,ZeroMeanFilteredOpen
Date,,,,,,,,
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,1455.219971,931800000,1509.891113,1528.983767
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1399.420044,1009000000,1460.423078,1528.983767
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1402.109985,1085500000,1438.658516,1528.983767
2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1403.449951,1092300000,1386.279016,1528.983767
2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1441.469971,1225200000,1399.262688,1528.983767


In [8]:
df_test['ZeroMeanFilteredOpen'] = df_mean
df_test.head()

,Open,High,Low,Close,Adj Close,Volume,Filtered_Open,ZeroMeanFilteredOpen
Date,,,,,,,,
2019-05-15,2820.379883,2858.679932,2815.080078,2850.959961,2850.959961,3125950000,2840.351719,1528.983767
2019-05-16,2855.800049,2892.149902,2855.800049,2876.320068,2876.320068,3338060000,2839.855921,1528.983767
2019-05-17,2858.600098,2885.479980,2854.229980,2859.530029,2859.530029,3257950000,2839.645614,1528.983767
2019-05-20,2841.939941,2853.860107,2831.290039,2840.229980,2840.229980,3288870000,2838.708026,1528.983767
2019-05-21,2854.020020,2868.879883,2854.020020,2864.360107,2864.360107,3218700000,2834.590994,1528.983767


Now, let's declare three variables containing a StandardScaler, without any additional parameters. 

We can't use the same scaler for the three Opens because we'll be using the scaler to un-scale predictions as well so the scaler properties must be specific to each Open data.

Then, we'll scale our column values and use that for training.

In [9]:
scaler = StandardScaler()
scaler.fit_transform(df_train.drop(columns = ['High', 'Low', 'Close', 'Adj Close', 'Volume', 'Filtered_Open', 'ZeroMeanFilteredOpen']))

array([[-0.1114101 ],
       [-0.13761036],
       [-0.24181344],
       ...,
       [ 2.41126057],
       [ 2.41174567],
       [ 2.47789075]])

In [10]:
filterScaler = StandardScaler()
filterScaler.fit_transform(df_train.drop(columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'ZeroMeanFilteredOpen']))

array([[-0.03568282],
       [-0.12813507],
       [-0.1688115 ],
       ...,
       [ 2.4588267 ],
       [ 2.45085386],
       [ 2.44992725]])

In [13]:
zeroScaler = StandardScaler()
zeroScaler.fit_transform(df_train.drop(columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Filtered_Open']))

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])